In [1]:
!pip install bs4

     |████████████████████████████████| 122kB 5.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
table = soup.find('table', {'class':'wikitable sortable'}).tbody
#print(table)

In [5]:
rows = table.find_all('tr')
columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]
print(columns)

['Postal Code', 'Borough', 'Neighbourhood']


In [6]:
print(rows[0:5])

[<tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>, <tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>]


In [26]:
df = pd.DataFrame(columns=columns)
for i in range(1,len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) < 3:
        values = [tds[0].text.replace('\n',''), tds[1].text.replace('\n',''), tds[2].text.replace('\n','')]
    else:
        values = [td.text.replace('\n','') for td in tds]
    df = df.append(pd.Series(values, index=columns), ignore_index=True) 

In [27]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [28]:
import numpy as np

In [29]:
# Remove rows with 'Not assigned' in their Borough attribute
df = df[df['Borough'].replace('Not assigned',np.nan).notnull()]

In [30]:
# Reset the index after removing some rows
df = df.reset_index(drop=True)

In [31]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Let's check if there is any repitition in the postal code:
#### If the 

In [32]:
pd.set_option("display.max_rows", None, "display.max_columns", None) 
# Show the counts of each unique postal code
df.groupby(['Postal Code'])['Postal Code'].value_counts()

Postal Code  Postal Code
M1B          M1B            1
M1C          M1C            1
M1E          M1E            1
M1G          M1G            1
M1H          M1H            1
M1J          M1J            1
M1K          M1K            1
M1L          M1L            1
M1M          M1M            1
M1N          M1N            1
M1P          M1P            1
M1R          M1R            1
M1S          M1S            1
M1T          M1T            1
M1V          M1V            1
M1W          M1W            1
M1X          M1X            1
M2H          M2H            1
M2J          M2J            1
M2K          M2K            1
M2L          M2L            1
M2M          M2M            1
M2N          M2N            1
M2P          M2P            1
M2R          M2R            1
M3A          M3A            1
M3B          M3B            1
M3C          M3C            1
M3H          M3H            1
M3J          M3J            1
M3K          M3K            1
M3L          M3L            1
M3M          M3

In [33]:
# If the number of the unique postal codes is equal to postal codes in the original dataframe, then all postal codes are unique
if df.shape[0] == df.groupby(['Postal Code'])['Postal Code'].value_counts().shape[0]:
    print('Every postal code in the data frame is unique')

Every postal code in the data frame is unique


### The dataframe shape (rows, columns) is:

In [34]:
# Show the shape of the cleaned dataframe
print(df.shape)

(103, 3)


### Adding latitudes and longitudes

In [35]:
import pandas as pd
lat_long_df = pd.read_csv('https://cocl.us/Geospatial_data')
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [46]:
# let's take the values of latitude and longitudes from the lat_long_df dataframe
df_new = df.merge(lat_long_df, on='Postal Code', how='left')
df_new.head(30)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
